- This program was used to extract sentences with promoters/inhibitors and causal verbs as described in the paper here ____. 
- See last cell for breakdown per molecule
- **Conclusion: Around 30% of sentences had molecules with causual verbs.**


# Acquiring All Papers into Array

In [ ]:
!pip install python-docx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.6 MB 3.7 MB/s 
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184507 sha256=a7928a1e15390ebeaf499e7d016efcde16f2fb055e1192ed89939b7bd0bc4b1e
  Stored in directory: /root/.cache/pip/wheels/f6/6f/b9/d798122a8b55b74ad30b5f52b01482169b445fbb84a11797a6
Successfully built python-docx


In [ ]:
!pip install glove_python-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 948 kB 4.1 MB/s 


In [ ]:
from docx import Document
import nltk
nltk.download('punkt')
import re
from nltk import sent_tokenize
import pandas as pd
from nltk.corpus import stopwords
nltk.download('stopwords')
import pickle
import numpy as np
import glob

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from nltk.stem import WordNetLemmatizer
import nltk 
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
!git clone 'https://github.com/igorbrigadir/stopwords.git'

Cloning into 'stopwords'...
remote: Enumerating objects: 149, done.
remote: Total 149 (delta 0), reused 0 (delta 0), pack-reused 149
Receiving objects: 100% (149/149), 85.27 KiB | 3.71 MiB/s, done.
Resolving deltas: 100% (52/52), done.


In [ ]:
alir3z4_data = '/content/stopwords/en/alir3z4.txt'

more_stops = pd.read_csv('/content/stopwords/en/alir3z4.txt')
new_stops = list(more_stops["'ll"])

In [ ]:
DOMAIN_STOPS = {'pubmed', 'et', 'al', 'page'}
STOPWORDS =  set(stopwords.words('english') + stopwords.words('german') +  stopwords.words('dutch') + stopwords.words('french') +  stopwords.words('spanish')  + new_stops) | DOMAIN_STOPS
STOPWORDS = set(STOPWORDS)

In [ ]:
len(STOPWORDS)

2011

In [ ]:
'a' in STOPWORDS

True

In [ ]:
ROOT = "/content/drive/MyDrive/regen_x"

In [ ]:
def get_docx(file_path):
    doc = []
    for para in Document(file_path).paragraphs:
        if para.text == "":
            continue
        doc += (sent_tokenize(para.text.lower())) # we lower text here
    return doc


def get_start_stop():
    domain_stops = {'pubmed', 'et', 'al', 'page'}
    with open('/content/stopwords/en/alir3z4.txt', 'r') as fn:
        new_stops = [line.strip() for line in fn.readlines()]
    STOPWORDS =  set(stopwords.words('english') + stopwords.words('german') +  stopwords.words('dutch') + stopwords.words('french') +  stopwords.words('spanish')  + new_stops) | domain_stops

    fn = glob.glob(ROOT + '/data/start-words/*')
    ALL_STARTS = [pickle.load(open(f , 'rb')) for f in fn]
    STARTWORDS = {}
    for f in ALL_STARTS:
      STARTWORDS.update(f)
    STARTWORDS = set(STARTWORDS.keys())

    assert(type(STOPWORDS)==set and type(STARTWORDS)==set)
    return (STARTWORDS, STOPWORDS)

In [ ]:
STARTWORDS, STOPWORDS = get_start_stop()

In [ ]:
from gensim.models import Word2Vec

In [ ]:
from natsort import natsorted
import os

In [ ]:
ROOT = "/content/drive/MyDrive/regen_x"

In [ ]:
# for lemmatization 
import spacy
# Initialize spacy 'en' model, keeping only tagger component needed for lemmatization
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [ ]:
def do_lemmatizing(filtered):
  # convert list to string 
  spacy_parsed_text = nlp(" ".join(filtered)) 
  # Get the lemma for each token in the parsed text 
  
  # I wanted to keep pronouns so not taking lemma if it's a pronoun but if you want to remove pronouns use below commented line 
  # return " ".join([token.lemma_ for token in doc])

  # return as list of words again 
  return [token.lemma_ if token.lemma_ != '-PRON-' else token.lower_ for token in spacy_parsed_text]

In [ ]:
# Uses lemmatization and stopwords 
def get_proc_docs_total(all_papers_path, STARTWORDS, STOPWORDS, max_papers=None, verbose=True):
  file_paths = all_papers_path

  print("Number of files: {}".format(len(file_paths)))
  if len(file_paths) == 0:
    # raise Exception("Folder has no files - maybe drive was not mounted?")
    pass 
  ## -- Collecting Papers from Given Year -- ##
  proc_docs = [] 

  counter = 1
  length = len(file_paths)
  for f in file_paths:
    doc = get_docx(f)
    
    for sentence in doc:
      # don't think we need to remove stopwords and such if we're training embeddings 
      # do lemmatization here as well 

      proc_sentence = [] 
      proc_sentence = [word for word in re.findall(r'\w+', sentence) if ((len(word) > 2) and (word not in STOPWORDS))]

      proc_sentence = do_lemmatizing(proc_sentence) 
      
      proc_docs.append(proc_sentence)  

    if(verbose):
      print("\t{}/{}".format(counter, length))
    counter += 1

    if max_papers != None:
      if counter == max_papers+1:
        break 

  return proc_docs

In [ ]:
def get_docs_total(all_papers_path, max_papers=None, verbose=True):
  file_paths = all_papers_path

  print("Number of files: {}".format(len(file_paths)))
  if len(file_paths) == 0:
    raise Exception("Folder has no files - maybe drive was not mounted?")
    pass 
  
  all_docs = [] 

  counter = 1
  length = len(file_paths)
  for f in file_paths:
    doc = get_docx(f)
    
    all_docs.append(doc)  

    if(verbose):
      print("\t{}/{}".format(counter, length))
    counter += 1

    if max_papers != None:
      if counter == max_papers+1:
        break 

  return all_docs

In [ ]:
all_papers_path = glob.glob(ROOT + '/data/ocr_paper_COMPREHENSIVE/*/*.docx')
all_papers_path[160:170]

['/content/drive/MyDrive/regen_x/data/ocr_paper_COMPREHENSIVE/1915/1915_R. H. Jocelyn Swan.docx',
 '/content/drive/MyDrive/regen_x/data/ocr_paper_COMPREHENSIVE/1915/1915_R. Ingebrigtsen.docx',
 '/content/drive/MyDrive/regen_x/data/ocr_paper_COMPREHENSIVE/1915/1915_The British Medical Journal.docx',
 '/content/drive/MyDrive/regen_x/data/ocr_paper_COMPREHENSIVE/1915/1915_T. N. Foulis.docx',
 '/content/drive/MyDrive/regen_x/data/ocr_paper_COMPREHENSIVE/1915/1915_W. Harris.docx',
 '/content/drive/MyDrive/regen_x/data/ocr_paper_COMPREHENSIVE/1916/1916_A. R. Robertson.docx',
 '/content/drive/MyDrive/regen_x/data/ocr_paper_COMPREHENSIVE/1916/1916_A.T. Mussen.docx',
 '/content/drive/MyDrive/regen_x/data/ocr_paper_COMPREHENSIVE/1916/1916_D. Orr.docx',
 '/content/drive/MyDrive/regen_x/data/ocr_paper_COMPREHENSIVE/1916/1916_Eijkman, C. .docx',
 '/content/drive/MyDrive/regen_x/data/ocr_paper_COMPREHENSIVE/1916/1916_J. COLLIER.docx']

In [ ]:
MODEL_PATH = "/content/drive/MyDrive/Colab Notebooks/NLP - Lab/WordEmbeddings/Models/Extracted Sentences/"

if not os.path.exists(MODEL_PATH):
    print("New path created: {}".format(MODEL_PATH))
    os.makedirs(MODEL_PATH)

In [ ]:
counter = 0 

# all_proc_docs = [] 

# Process the docs (stopwords and lemmatization)
# all_proc_docs = get_proc_docs_total(all_papers_path, STARTWORDS, STOPWORDS, max_papers=None, verbose=True)

In [ ]:
# TODO: Try with processed docs 

# Unprocessed all docs: 
all_docs = get_docs_total(all_papers_path)

In [ ]:
len(all_docs)

700

In [ ]:
import json 

with open(MODEL_PATH + "all_years_array.json", "w") as f:
    json.dump(all_docs, f)

# Extracting Sentences with Causal Verbs

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
low_ambig_high_freq = ["give rise (to)", "induce", "produce", "generate", "effect", "bring about", "provoke", "arouse", "elicit", "lead (to)", "trigger", "derive (from)", "associate (with)", "relate (to)",
                "link (to)", "stem (from)", "originate", "bring forth", "lead up", "trigger off", "bring on", "result (from)" ]

In [ ]:
low_ambig_low_freq = ["stir up", "entail", "contribute (to)", "set up", "trigger off", "commence", "set off", "set in motion", "bring on", "conduce (to)", "educe", "originate in",
                      "lead off", "spark", "spark off", "evoke", "link up", "implicate (in)", "activate", "actuate", "kindle", "fire up"] 

In [ ]:
high_ambig_low_freq = ["create", "launch", "develop", "bring", "stimulate", "call forth", "unleash", "effectuate", "kick up", "give birth (to)", "call down", "put forward", "cause"]

In [ ]:
high_ambig_high_freq = ["start", "make", "begin", "rise"]

In [ ]:
causal_verbs = low_ambig_high_freq + low_ambig_low_freq + high_ambig_low_freq + high_ambig_high_freq

In [ ]:
print(causal_verbs)

['give rise (to)', 'induce', 'produce', 'generate', 'effect', 'bring about', 'provoke', 'arouse', 'elicit', 'lead (to)', 'trigger', 'derive (from)', 'associate (with)', 'relate (to)', 'link (to)', 'stem (from)', 'originate', 'bring forth', 'lead up', 'trigger off', 'bring on', 'result (from)', 'stir up', 'entail', 'contribute (to)', 'set up', 'trigger off', 'commence', 'set off', 'set in motion', 'bring on', 'conduce (to)', 'educe', 'originate in', 'lead off', 'spark', 'spark off', 'evoke', 'link up', 'implicate (in)', 'activate', 'actuate', 'kindle', 'fire up', 'create', 'launch', 'develop', 'bring', 'stimulate', 'call forth', 'unleash', 'effectuate', 'kick up', 'give birth (to)', 'call down', 'put forward', 'cause', 'start', 'make', 'begin', 'rise']


In [ ]:
# remove parenthesis
import re
causal_verbs = [re.sub(r'[()]', '', word) for word in causal_verbs]
print(causal_verbs)

['give rise to', 'induce', 'produce', 'generate', 'effect', 'bring about', 'provoke', 'arouse', 'elicit', 'lead to', 'trigger', 'derive from', 'associate with', 'relate to', 'link to', 'stem from', 'originate', 'bring forth', 'lead up', 'trigger off', 'bring on', 'result from', 'stir up', 'entail', 'contribute to', 'set up', 'trigger off', 'commence', 'set off', 'set in motion', 'bring on', 'conduce to', 'educe', 'originate in', 'lead off', 'spark', 'spark off', 'evoke', 'link up', 'implicate in', 'activate', 'actuate', 'kindle', 'fire up', 'create', 'launch', 'develop', 'bring', 'stimulate', 'call forth', 'unleash', 'effectuate', 'kick up', 'give birth to', 'call down', 'put forward', 'cause', 'start', 'make', 'begin', 'rise']


In [ ]:
MODEL_PATH = "/content/drive/MyDrive/Colab Notebooks/NLP - Lab/WordEmbeddings/Models/Extracted Sentences/"

In [ ]:
import json

In [ ]:
with open(MODEL_PATH + "all_years_array.json", "r") as f:
    all_docs = json.load(f)

In [ ]:
len(all_docs)

700

In [ ]:
all_docs_1D = [j for sub in all_docs for j in sub]

In [ ]:
len(all_docs_1D)

755496

In [ ]:
keyword = "pten"
found_sentences = [] 

for sentence in all_docs_1D:
  if keyword in sentence:
    found_sentences.append(sentence)

In [ ]:
len(found_sentences)

660

In [ ]:
found_sentences

["a£$iptenzarzt an der univc'rsitfttsklinik zn konig herg· i. pr.",
 'man wurde nach allem diesern mit einigem recht behaupten  konnen, dass die zwischensubstanz sehr leicht gerinnbar ist.',
 'ich 5     behaupten  mlissen,  dass  sie  nicht  dem entwickelungsmodus',
 'was die graue substanz betrifft, so ltisst es sich kaum bebaupten, dass gerade in einem bestimmten  monat  die  gefassneubildung in ihr eine besonders lebhafte ist.',
 'jedoch muss ich gegen riede i fiir das jugendliche riickenmark  wenigstens behaupten,  dass  in der  wand',
 'auch muss ich go i g i 1 ) und bo ii gcgcnliber behaupten, was ich auch von den bei der regeneration sich ent\xad wickelnden gliazellen getlian babe, dass verastelungen der fort\xad satze,  wenn  auch  selten   vorkommen.',
 "so sehr ich die grossen verdienste dcr untersuchungen von jastrowitz anerkenne und hochschatze, so rnochte ich doch be\xad haupten, dass die grenze zwischen den normalen und pathologischen veranderungen zu ungunsten der letzte

In [ ]:
keyword_dict = {} 
keywords = ["netrin", "ephrin", "laminin", "tenascin", "cspg", "zymosan", "camp", "pten", "cntf", "lif", "oncomodulin", "stat3", "socs3", "rhoa", "rock", "y27632", "nogo", "klf", "ngr", "lar", "tlr2", "bdnf", "igf1", "opn", "mag", "omgp", "kspg", "taxol"]
#keywords = ["nucleus"]
for keyword in keywords:
  found_sentences = [] 

  for sentence in all_docs_1D:
    if keyword in sentence:
      found_sentences.append(sentence)

  counter = 0
  for sentence in found_sentences:
    if any(word in sentence for word in causal_verbs):
      # print("-> " + sentence)
      counter += 1

  to_append = [str(int(counter/len(found_sentences) * 100)) + "%", int(counter), int(len(found_sentences))]
  keyword_dict[keyword] = to_append
  print("{} {}/{}: {}".format(keyword, counter, len(found_sentences), counter / len(found_sentences)))

netrin 38/183: 0.20765027322404372
ephrin 72/242: 0.2975206611570248
laminin 189/913: 0.20700985761226726
tenascin 122/511: 0.23874755381604695
cspg 229/869: 0.26352128883774456
zymosan 28/59: 0.4745762711864407
camp 336/1664: 0.20192307692307693
pten 184/660: 0.2787878787878788
cntf 182/517: 0.3520309477756286
lif 899/4896: 0.18361928104575165
oncomodulin 11/47: 0.23404255319148937
stat3 71/255: 0.2784313725490196
socs3 145/352: 0.4119318181818182
rhoa 116/334: 0.3473053892215569
rock 128/697: 0.1836441893830703
y27632 3/4: 0.75
nogo 262/1592: 0.16457286432160803
klf 35/171: 0.2046783625730994
ngr 214/1391: 0.15384615384615385
lar 5285/38353: 0.13779886840664354
tlr2 1/3: 0.3333333333333333
bdnf 310/1027: 0.3018500486854917
igf1 9/55: 0.16363636363636364
opn 32/165: 0.19393939393939394
mag 1108/6690: 0.16562032884902841
omgp 16/132: 0.12121212121212122
kspg 0/5: 0.0
taxol 2/17: 0.11764705882352941


In [ ]:
import pandas as pd 
df = pd.DataFrame(keyword_dict)
df = df.T # flip columns with rows 
df.columns = ["% sentences with causal verb", "sentences with molecule and causal verb", "total sentences with molecule"]

In [ ]:
df

,% sentences with causal verb,sentences with molecule and causal verb,total sentences with molecule
netrin,20%,38,183
ephrin,29%,72,242
laminin,20%,189,913
tenascin,23%,122,511
cspg,26%,229,869
zymosan,47%,28,59
camp,20%,336,1664
pten,27%,184,660
cntf,35%,182,517
lif,18%,899,4896
